###########################################################

이 부분은 챗봇에 모델이 있어서 일단 올리기는 하는데
기존에 모델과 같아서 필요 없다고 생각하면 빼도 될 것 같습니다.

#############################################################

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import sqlite3

In [ ]:
# 건물 인코딩.csv
# 강의실 건물별 TOP3 최종.csv
# all.tar
# model.pt
# model_state_dict.pt

# 위의 5개 파일을 불러와야 합니다

encoding_df = pd.read_csv('건물 인코딩.csv')
classroom = pd.read_csv('강의실 건물별 TOP3 최종.csv')
classroom_drop_room = classroom.drop(['room'], axis =1)

# classroom encoding (건물이름 --> 숫자)
classroom_encode = pd.merge(classroom_drop_room, encoding_df, on= 'building', how ='left')
classroom_encode = classroom_encode.drop(['building'], axis =1)
classroom_encode = classroom_encode.rename({'encoding':'building'},axis=1)
classroom_encode = classroom_encode.to_numpy()

# 모델 --> 없으면 모델을 불러올 수가 없어요
class NNCollabFiltering(nn.Module):
    def __init__(self, emb_size=10, n_hidden=100):
        super(NNCollabFiltering, self).__init__()
        self.emb_size = emb_size
        self.user_emb = nn.Embedding(100, emb_size)
        self.item_emb = nn.Embedding(600, emb_size)
        self.lin1 = nn.Linear(1000, n_hidden)
        self.lin2 = nn.Linear(n_hidden, 1)
        self.drop1 = nn.Dropout(0.1)
        
    def forward(self, u, v):
        U = self.user_emb(u)
        U = torch.reshape(U, (82, 4 * self.emb_size))
        V = self.item_emb(v)
        V = torch.reshape(V, (82,6 * self.emb_size))
        x = F.relu(torch.cat([U, V], dim=1))
        x = self.drop1(x)
        x = F.relu(self.lin1(x))
        x = self.lin2(x)
        x = torch.flatten(x)
        return x


# 모델 불러오기
PATH = ''
model = torch.load(PATH + 'model.pt')  # 전체 모델을 통째로 불러옴, 클래스 선언 필수 (위의 모델 코드 있어야 함)
model.load_state_dict(torch.load(PATH + 'model_state_dict.pt'))  # state_dict를 불러 온 후, 모델에 저장
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay= 1e-6)

checkpoint = torch.load(PATH + 'all.tar')   # dict 불러오기
model.load_state_dict(checkpoint['model'])
optimizer.load_state_dict(checkpoint['optimizer'])

def softmax(a) :
    exp_a = np.exp(a)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y

# 해당 시간대에 예약이 모두 찬 강의실 DB를 통해 확인하여 반환
def get_full_classrooms(time):
  full_list = []
  starttime = time[0]
  endtime = time[1]
  
  conn = sqlite3.connect("gmc.db")
  cur = conn.cursor()
  cur.execute("SELECT building, name, SUM (member_num), starttime, endtime FROM classroom_reservation WHERE (starttime>'" + starttime + "' AND starttime<'" + endtime + "') OR (endtime>'" + starttime + "' AND endtime<'" + endtime + "') GROUP BY building, name;")
  
  rows = cur.fetchall()
  print(rows)

  for row in rows:
    cur.execute("SELECT capacity FROM classroom WHERE building='"+row[0]+"' AND name='"+row[1]+"';")
    cap = cur.fetchall()[0][0]
    if cap <= row[2]:
      full_list.append((row[0], row[1]))
  
  conn.close()
  return full_list

###############       9개 추천해주는 모델      ########################
def classroom_Top9(num_user, time, location, plug, table, chair):
  
  # 인원 수, 시간 정보 사용 안함 
  new_user = [plug, table, chair, location]
  building_row = encoding_df.loc[encoding_df['building'] == location]
  encoding = building_row.iloc[0]['encoding']
  new_user[3] = encoding

  with torch.no_grad():
    user_scale = np.array(new_user*82).reshape((82,4))
    user_input = torch.LongTensor(user_scale)
    classroom_input = torch.LongTensor(classroom_encode[:,1:])
    out = model(user_input, classroom_input).numpy()
    sort_out = np.argsort(out)[::-1]
    sort_out_score = out[sort_out]
  
  # 예약이 모두 찬 강의실 확인 및 제외
  full_list = get_full_classrooms(time)
  full_indices = []
  for fc in full_list:
    filtered = classroom[classroom['building'] == fc[0]]
    filtered = filtered[filtered['room'] == fc[1]]
    full_inx = filtered.classroom_index.tolist()[0]
    rmv = sort_out.tolist().index(full_inx)
    sort_out = np.delete(sort_out, rmv)
    sort_out_score = np.delete(sort_out_score, rmv)
    full_indices.append(full_inx)
  
  print(sort_out)
  # Softmax 추천 알고리즘
  top9_index = np.random.choice(sort_out, 9, p=softmax(sort_out_score), replace=False)
  print(top9_index)
  
  top9_df = pd.DataFrame(top9_index, columns =['classroom_index'])
  top9 = pd.merge(top9_df, classroom, on= 'classroom_index', how ='left')
  output = top9[['building', 'room']].copy().to_records(index=False)
    
  return output 
##############################################################################################


result = classroom_Top9(3, ('2021-08-09T15:00:00+09:00', '2021-08-09T16:00:00+09:00'), '국제관', 3, 1, 0)
print(result)